<a href="https://colab.research.google.com/github/PrateekCoder/NLP_Zero_To_Hero/blob/main/NLP_Zero_to_Hero.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [18]:
import pandas as pd

In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install --upgrade gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
train = pd.read_csv('gdrive/My Drive/datasets/imdb_sentiment_analysis/Train.csv')
test = pd.read_csv('gdrive/My Drive/datasets/imdb_sentiment_analysis/Test.csv')
valid = pd.read_csv('gdrive/My Drive/datasets/imdb_sentiment_analysis/Valid.csv')

In [6]:
df = pd.read_csv('gdrive/My Drive/datasets/imdb_sentiment_analysis/IMDB Dataset.csv')

## Text Pre-Processing

In [7]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [8]:
# Download required NLTK resources if not already downloaded
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [20]:
def preprocess_text(text):
    #print("Original Text:", text)

    # Remove HTML tags
    text = re.sub('<[^>]*>', '', text)

    print("Text after removing HTML tags:", text)

    # Remove non-alphabetic characters and convert to lowercase
    text = re.sub('[^a-zA-Z]', ' ', text).lower()

    print("Text after removing non-alphabetic characters and converting to lowercase:", text)

    # Tokenize the text
    words = word_tokenize(text)

    print("Text after tokenization:", words)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]

    print("Text after removing stop words:", words)

    # Lemmatize the words
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]

    print("Text after Lemmatization:", words)

    # Combine words back into a single string
    preprocessed_text = ' '.join(words)

    print("Final pre-processed text:", preprocessed_text)

    return preprocessed_text

In [21]:
preprocess_text("This is Prateek Gaurav, I am making this video:, related to <NLP>. I WILL BE explaining all the techniques used in NLP and about RNN.")

Text after removing HTML tags: This is Prateek Gaurav, I am making this video:, related to . I WILL BE explaining all the techniques used in NLP and about RNN.
Text after removing non-alphabetic characters and converting to lowercase: this is prateek gaurav  i am making this video   related to   i will be explaining all the techniques used in nlp and about rnn 
Text after tokenization: ['this', 'is', 'prateek', 'gaurav', 'i', 'am', 'making', 'this', 'video', 'related', 'to', 'i', 'will', 'be', 'explaining', 'all', 'the', 'techniques', 'used', 'in', 'nlp', 'and', 'about', 'rnn']
Text after removing stop words: ['prateek', 'gaurav', 'making', 'video', 'related', 'explaining', 'techniques', 'used', 'nlp', 'rnn']
Text after Lemmatization: ['prateek', 'gaurav', 'making', 'video', 'related', 'explaining', 'technique', 'used', 'nlp', 'rnn']
Final pre-processed text: prateek gaurav making video related explaining technique used nlp rnn


'prateek gaurav making video related explaining technique used nlp rnn'

## Pre-Processing Our Data

In [11]:
%%time
train['preprocessed_text'] = train['text'].apply(preprocess_text)
test['preprocessed_text'] = test['text'].apply(preprocess_text)
valid['preprocessed_text'] = valid['text'].apply(preprocess_text)
df['preprocessed_text'] = df['review'].apply(preprocess_text)

CPU times: user 2min 21s, sys: 2.33 s, total: 2min 23s
Wall time: 2min 22s


In [22]:
train

,text,label,preprocessed_text
0,I grew up (b. 1965) watching and loving the Th...,0,grew b watching loving thunderbird mate school...
1,"When I put this movie in my DVD player, and sa...",0,put movie dvd player sat coke chip expectation...
2,Why do people who do not know what a particula...,0,people know particular time past like feel nee...
3,Even though I have great interest in Biblical ...,0,even though great interest biblical movie bore...
4,Im a die hard Dads Army fan and nothing will e...,1,im die hard dad army fan nothing ever change g...
...,...,...,...
39995,"""Western Union"" is something of a forgotten cl...",1,western union something forgotten classic west...
39996,This movie is an incredible piece of work. It ...,1,movie incredible piece work explores every noo...
39997,My wife and I watched this movie because we pl...,0,wife watched movie plan visit sicily stromboli...
39998,"When I first watched Flatliners, I was amazed....",1,first watched flatliners amazed necessary feat...


## Using BOW, Tf-IDF & Word2Vec with SVM

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from gensim.models import Word2Vec

def train_svm_with_representations(train_data, test_data, representation):
    if representation == 'bow':
        vectorizer = CountVectorizer()
    elif representation == 'tfidf':
        vectorizer = TfidfVectorizer()
    else:
        raise ValueError("Invalid representation. Choose 'bow' or 'tfidf'.")

    X_train = vectorizer.fit_transform(train_data)
    X_test = vectorizer.transform(test_data)

    clf = SVC()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    return y_pred

def get_word2vec_embeddings(data):
    tokenized_sentences = [sentence.split() for sentence in data]
    model = Word2Vec(tokenized_sentences, vector_size=100, window=5, min_count=1, workers=4)
    embeddings = np.array([np.mean([model.wv[word] for word in sentence], axis=0) for sentence in tokenized_sentences])

    return embeddings

def train_svm_with_word2vec(train_data, test_data):
    X_train = get_word2vec_embeddings(train_data)
    X_test = get_word2vec_embeddings(test_data)

    clf = SVC()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    return y_pred

In [ ]:
X_train = train['preprocessed_text']
y_train = train['label']
X_test = test['preprocessed_text']
y_test = test['label']

In [ ]:
%%time
# Bag of Words
y_pred_bow = train_svm_with_representations(X_train, X_test, 'bow')
accuracy_bow = accuracy_score(y_test, y_pred_bow)
print(accuracy_bow)

0.8804
CPU times: user 50min 5s, sys: 3.78 s, total: 50min 9s
Wall time: 50min 20s


In [ ]:
%%time
# TF-IDF
y_pred_tfidf = train_svm_with_representations(X_train, X_test, 'tfidf')
accuracy_tfidf = accuracy_score(y_test, y_pred_tfidf)
print(accuracy_tfidf)

0.9004
CPU times: user 1h 6min 58s, sys: 3.39 s, total: 1h 7min 1s
Wall time: 1h 7min 21s


In [ ]:
%%time
# Word2Vec
y_pred_word2vec = train_svm_with_word2vec(X_train, X_test)
accuracy_word2vec = accuracy_score(y_test, y_pred_word2vec)
print(accuracy_word2vec)

0.4992
CPU times: user 3min 43s, sys: 1.09 s, total: 3min 44s
Wall time: 3min 15s


The Word2Vec models performed very bad compared to BOW or TF-IDF, but theoretically it should perform better that bow and tf-idf, this is because we will need to train our word2vec model in a better way, try more data, try to play with different hyperparameters. This is computationally expensive process and it is always a good idea to use pretrained models. I will be using Google News 300 pretrained model here and see how that performs.

In [ ]:
import gensim.downloader as api

def get_google_word2vec_embeddings(data):
    # Load the Google News Word2Vec model
    model = api.load("word2vec-google-news-300")

    tokenized_sentences = [sentence.split() for sentence in data]
    embeddings = []

    for sentence in tokenized_sentences:
        sentence_embeddings = []
        for word in sentence:
            if word in model:
                sentence_embeddings.append(model[word])
        if sentence_embeddings:
            embeddings.append(np.mean(sentence_embeddings, axis=0))
        else:
            embeddings.append(np.zeros(300))

    return np.array(embeddings)

In [ ]:
def train_svm_with_google_word2vec(train_data, test_data):
    X_train = get_google_word2vec_embeddings(train_data)
    X_test = get_google_word2vec_embeddings(test_data)

    clf = SVC()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    return y_pred

In [ ]:
# Google News Word2Vec
y_pred_google_word2vec = train_svm_with_google_word2vec(X_train, X_test)
accuracy_google_word2vec = accuracy_score(y_test, y_pred_google_word2vec)
print(accuracy_google_word2vec)

[=================================================-] 99.7% 1657.5/1662.8MB downloaded
0.8574


## Vanilla RNN Model

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# Preparing the data
max_words = 10000
max_len = 100
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train['preprocessed_text'])

X_train = tokenizer.texts_to_sequences(train['preprocessed_text'])
X_test = tokenizer.texts_to_sequences(test['preprocessed_text'])
X_valid = tokenizer.texts_to_sequences(valid['preprocessed_text'])

X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)
X_valid = pad_sequences(X_valid, maxlen=max_len)

y_train = to_categorical(train['label'])
y_test = to_categorical(test['label'])
y_valid = to_categorical(valid['label'])

# Build the model
model = Sequential()
model.add(Embedding(max_words, 128, input_length=max_len))
model.add(SimpleRNN(64))
model.add(Dense(2, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=25, batch_size=128)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)

print("Test accuracy:", accuracy)

Epoch 1/25
313/313 [==============================] - 34s 102ms/step - loss: 0.4548 - accuracy: 0.7643 - val_loss: 0.3274 - val_accuracy: 0.8680
Epoch 2/25
313/313 [==============================] - 26s 84ms/step - loss: 0.2462 - accuracy: 0.9033 - val_loss: 0.3395 - val_accuracy: 0.8732
Epoch 3/25
313/313 [==============================] - 26s 84ms/step - loss: 0.1336 - accuracy: 0.9509 - val_loss: 0.4242 - val_accuracy: 0.8520
Epoch 4/25
313/313 [==============================] - 28s 90ms/step - loss: 0.0427 - accuracy: 0.9866 - val_loss: 0.5871 - val_accuracy: 0.8242
Epoch 5/25
313/313 [==============================] - 25s 80ms/step - loss: 0.0184 - accuracy: 0.9948 - val_loss: 0.6792 - val_accuracy: 0.8214
Epoch 6/25
313/313 [==============================] - 28s 90ms/step - loss: 0.0121 - accuracy: 0.9968 - val_loss: 0.7025 - val_accuracy: 0.8452
Epoch 7/25
313/313 [==============================] - 26s 84ms/step - loss: 0.0152 - accuracy: 0.9953 - val_loss: 0.7363 - val_accuracy

Testing with DF which is not already split adds a dropout layer after the RNN layer, which helps to prevent overfitting. The model is also set up to use early stopping based on validation loss, which will stop training if the validation loss does not improve for a certain number of epochs.

In [18]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder

# Label encode the target variable
le = LabelEncoder()
df['sentiment'] = le.fit_transform(df['sentiment'])

# Split into train, test and validation sets
X_train, X_test, y_train, y_test = train_test_split(df['review'], df['sentiment'], test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Tokenize the data
max_words = 10000
max_len = 100
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
X_valid = tokenizer.texts_to_sequences(X_valid)

X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)
X_valid = pad_sequences(X_valid, maxlen=max_len)

# Create the model
model = Sequential()
model.add(Embedding(max_words, 128, input_length=max_len))
model.add(SimpleRNN(64))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Set up early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(X_train, to_categorical(y_train), validation_data=(X_valid, to_categorical(y_valid)), 
                    epochs=25, batch_size=128, callbacks=[early_stop])

# Evaluate the model
loss, accuracy = model.evaluate(X_test, to_categorical(y_test))

print("Test accuracy:", accuracy)

Epoch 1/25
250/250 [==============================] - 9s 31ms/step - loss: 0.4980 - accuracy: 0.7410 - val_loss: 0.3522 - val_accuracy: 0.8533
Epoch 2/25
250/250 [==============================] - 7s 29ms/step - loss: 0.2956 - accuracy: 0.8812 - val_loss: 0.3383 - val_accuracy: 0.8614
Epoch 3/25
250/250 [==============================] - 7s 29ms/step - loss: 0.1889 - accuracy: 0.9307 - val_loss: 0.4156 - val_accuracy: 0.8461
Epoch 4/25
250/250 [==============================] - 7s 29ms/step - loss: 0.0877 - accuracy: 0.9703 - val_loss: 0.5292 - val_accuracy: 0.8303
Epoch 5/25
250/250 [==============================] - 7s 29ms/step - loss: 0.0487 - accuracy: 0.9831 - val_loss: 0.6432 - val_accuracy: 0.8261
Epoch 6/25
250/250 [==============================] - 7s 30ms/step - loss: 0.0381 - accuracy: 0.9868 - val_loss: 0.6976 - val_accuracy: 0.8346
Epoch 7/25
250/250 [==============================] - 7s 29ms/step - loss: 0.0404 - accuracy: 0.9859 - val_loss: 0.7521 - val_accuracy: 0.8195

## LSTM Model

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# Preparing the data
max_words = 10000
max_len = 100
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train['preprocessed_text'])

X_train = tokenizer.texts_to_sequences(train['preprocessed_text'])
X_test = tokenizer.texts_to_sequences(test['preprocessed_text'])
X_valid = tokenizer.texts_to_sequences(valid['preprocessed_text'])

X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)
X_valid = pad_sequences(X_valid, maxlen=max_len)

y_train = to_categorical(train['label'])
y_test = to_categorical(test['label'])
y_valid = to_categorical(valid['label'])

# Build the model
model = Sequential()
model.add(Embedding(max_words, 128, input_length=max_len))
model.add(LSTM(64))
model.add(Dense(2, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=10, batch_size=128)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)

print("Test accuracy:", accuracy)

Epoch 1/10
313/313 [==============================] - 85s 264ms/step - loss: 0.3573 - accuracy: 0.8403 - val_loss: 0.3008 - val_accuracy: 0.8754
Epoch 2/10
313/313 [==============================] - 81s 259ms/step - loss: 0.2191 - accuracy: 0.9153 - val_loss: 0.2933 - val_accuracy: 0.8840
Epoch 3/10
313/313 [==============================] - 83s 265ms/step - loss: 0.1615 - accuracy: 0.9393 - val_loss: 0.3346 - val_accuracy: 0.8768
Epoch 4/10
313/313 [==============================] - 82s 263ms/step - loss: 0.1147 - accuracy: 0.9581 - val_loss: 0.3781 - val_accuracy: 0.8688
Epoch 5/10
313/313 [==============================] - 82s 263ms/step - loss: 0.0892 - accuracy: 0.9690 - val_loss: 0.4818 - val_accuracy: 0.8654
Epoch 6/10
313/313 [==============================] - 85s 272ms/step - loss: 0.0697 - accuracy: 0.9766 - val_loss: 0.5939 - val_accuracy: 0.8604
Epoch 7/10
313/313 [==============================] - 82s 262ms/step - loss: 0.0471 - accuracy: 0.9844 - val_loss: 0.6234 - val_ac

Testing with DF which is not already split adds a dropout layer after the RNN layer, which helps to prevent overfitting. The model is also set up to use early stopping based on validation loss, which will stop training if the validation loss does not improve for a certain number of epochs.

In [21]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping

# Split into train, test and validation sets
X_train, X_test, y_train, y_test = train_test_split(df['review'], df['sentiment'], test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Tokenize the data
max_words = 10000
max_len = 100
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
X_valid = tokenizer.texts_to_sequences(X_valid)

X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)
X_valid = pad_sequences(X_valid, maxlen=max_len)

# Create the model
model = Sequential()
model.add(Embedding(max_words, 128, input_length=max_len))
model.add(LSTM(64))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Set up early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(X_train, to_categorical(y_train), validation_data=(X_valid, to_categorical(y_valid)),
                    epochs=10, batch_size=128, callbacks=[early_stop])

# Evaluate the model
loss, accuracy = model.evaluate(X_test, to_categorical(y_test))

print("Test accuracy:", accuracy)

Epoch 1/10
250/250 [==============================] - 22s 83ms/step - loss: 0.4161 - accuracy: 0.7999 - val_loss: 0.3090 - val_accuracy: 0.8719
Epoch 2/10
250/250 [==============================] - 20s 81ms/step - loss: 0.2656 - accuracy: 0.8940 - val_loss: 0.3153 - val_accuracy: 0.8654
Epoch 3/10
250/250 [==============================] - 21s 84ms/step - loss: 0.2047 - accuracy: 0.9207 - val_loss: 0.3478 - val_accuracy: 0.8600
Epoch 4/10
250/250 [==============================] - 21s 82ms/step - loss: 0.1581 - accuracy: 0.9404 - val_loss: 0.4014 - val_accuracy: 0.8601
Epoch 5/10
250/250 [==============================] - 21s 84ms/step - loss: 0.1210 - accuracy: 0.9560 - val_loss: 0.5088 - val_accuracy: 0.8510
Epoch 6/10
313/313 [==============================] - 4s 14ms/step - loss: 0.3134 - accuracy: 0.8640
Test accuracy: 0.8640000224113464


## GRU Model

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# Preparing the data
max_words = 10000
max_len = 100
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train['preprocessed_text'])

X_train = tokenizer.texts_to_sequences(train['preprocessed_text'])
X_test = tokenizer.texts_to_sequences(test['preprocessed_text'])
X_valid = tokenizer.texts_to_sequences(valid['preprocessed_text'])

X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)
X_valid = pad_sequences(X_valid, maxlen=max_len)

y_train = to_categorical(train['label'])
y_test = to_categorical(test['label'])
y_valid = to_categorical(valid['label'])

# Build the model
model = Sequential()
model.add(Embedding(max_words, 128, input_length=max_len))
model.add(GRU(64))
model.add(Dense(2, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=10, batch_size=128)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)

print("Test accuracy:", accuracy)

Epoch 1/10
313/313 [==============================] - 90s 277ms/step - loss: 0.3725 - accuracy: 0.8233 - val_loss: 0.2882 - val_accuracy: 0.8820
Epoch 2/10
313/313 [==============================] - 80s 256ms/step - loss: 0.2246 - accuracy: 0.9133 - val_loss: 0.2915 - val_accuracy: 0.8832
Epoch 3/10
313/313 [==============================] - 77s 246ms/step - loss: 0.1671 - accuracy: 0.9362 - val_loss: 0.3415 - val_accuracy: 0.8764
Epoch 4/10
313/313 [==============================] - 78s 248ms/step - loss: 0.1237 - accuracy: 0.9555 - val_loss: 0.3938 - val_accuracy: 0.8668
Epoch 5/10
313/313 [==============================] - 79s 253ms/step - loss: 0.0938 - accuracy: 0.9672 - val_loss: 0.4272 - val_accuracy: 0.8670
Epoch 6/10
313/313 [==============================] - 79s 252ms/step - loss: 0.0757 - accuracy: 0.9747 - val_loss: 0.5267 - val_accuracy: 0.8646
Epoch 7/10
313/313 [==============================] - 80s 254ms/step - loss: 0.0615 - accuracy: 0.9794 - val_loss: 0.5645 - val_ac

Testing with DF which is not already split adds a dropout layer after the RNN layer, which helps to prevent overfitting. The model is also set up to use early stopping based on validation loss, which will stop training if the validation loss does not improve for a certain number of epochs.

In [22]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Embedding, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping

# Split into train, test and validation sets
X_train, X_test, y_train, y_test = train_test_split(df['review'], df['sentiment'], test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Tokenize the data
max_words = 10000
max_len = 100
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
X_valid = tokenizer.texts_to_sequences(X_valid)

X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)
X_valid = pad_sequences(X_valid, maxlen=max_len)

# Create the model
model = Sequential()
model.add(Embedding(max_words, 128, input_length=max_len))
model.add(GRU(64))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Set up early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
 
# Train the model
history = model.fit(X_train, to_categorical(y_train), validation_data=(X_valid, to_categorical(y_valid)), 
                    epochs=25, batch_size=128, callbacks=[early_stop])

# Evaluate the model
loss, accuracy = model.evaluate(X_test, to_categorical(y_test))

print("Test accuracy:", accuracy)

Epoch 1/25
250/250 [==============================] - 20s 72ms/step - loss: 0.4588 - accuracy: 0.7679 - val_loss: 0.3179 - val_accuracy: 0.8629
Epoch 2/25
250/250 [==============================] - 18s 70ms/step - loss: 0.2699 - accuracy: 0.8907 - val_loss: 0.3211 - val_accuracy: 0.8680
Epoch 3/25
250/250 [==============================] - 17s 69ms/step - loss: 0.2041 - accuracy: 0.9227 - val_loss: 0.3316 - val_accuracy: 0.8608
Epoch 4/25
250/250 [==============================] - 18s 70ms/step - loss: 0.1561 - accuracy: 0.9426 - val_loss: 0.4234 - val_accuracy: 0.8551
Epoch 5/25
250/250 [==============================] - 18s 70ms/step - loss: 0.1186 - accuracy: 0.9571 - val_loss: 0.4981 - val_accuracy: 0.8518
Epoch 6/25
313/313 [==============================] - 3s 10ms/step - loss: 0.3258 - accuracy: 0.8560
Test accuracy: 0.8560000061988831


## Bi-Directional LSTM

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# Preparing the data
max_words = 10000
max_len = 100
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train['preprocessed_text'])

X_train = tokenizer.texts_to_sequences(train['preprocessed_text'])
X_test = tokenizer.texts_to_sequences(test['preprocessed_text'])
X_valid = tokenizer.texts_to_sequences(valid['preprocessed_text'])

X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)
X_valid = pad_sequences(X_valid, maxlen=max_len)

y_train = to_categorical(train['label'])
y_test = to_categorical(test['label'])
y_valid = to_categorical(valid['label'])

# Build the model
model = Sequential()
model.add(Embedding(max_words, 128, input_length=max_len))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(2, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=10, batch_size=128)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)

print("Test accuracy:", accuracy)

Epoch 1/10
313/313 [==============================] - 32s 91ms/step - loss: 0.3596 - accuracy: 0.8391 - val_loss: 0.2865 - val_accuracy: 0.8830
Epoch 2/10
313/313 [==============================] - 27s 86ms/step - loss: 0.2197 - accuracy: 0.9141 - val_loss: 0.3032 - val_accuracy: 0.8822
Epoch 3/10
313/313 [==============================] - 27s 88ms/step - loss: 0.1634 - accuracy: 0.9373 - val_loss: 0.3316 - val_accuracy: 0.8770
Epoch 4/10
313/313 [==============================] - 27s 88ms/step - loss: 0.1203 - accuracy: 0.9557 - val_loss: 0.4034 - val_accuracy: 0.8734
Epoch 5/10
313/313 [==============================] - 28s 88ms/step - loss: 0.0892 - accuracy: 0.9690 - val_loss: 0.4922 - val_accuracy: 0.8674
Epoch 6/10
313/313 [==============================] - 28s 89ms/step - loss: 0.0604 - accuracy: 0.9790 - val_loss: 0.5353 - val_accuracy: 0.8652
Epoch 7/10
313/313 [==============================] - 28s 88ms/step - loss: 0.0452 - accuracy: 0.9847 - val_loss: 0.5945 - val_accuracy:

Testing with DF which is not already split adds a dropout layer after the RNN layer, which helps to prevent overfitting. The model is also set up to use early stopping based on validation loss, which will stop training if the validation loss does not improve for a certain number of epochs.

In [26]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping

# Split into train, test and validation sets
X_train, X_test, y_train, y_test = train_test_split(df['review'], df['sentiment'], test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Tokenize the data
max_words = 10000
max_len = 100
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
X_valid = tokenizer.texts_to_sequences(X_valid)

X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)
X_valid = pad_sequences(X_valid, maxlen=max_len)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_valid = to_categorical(y_valid)

# Build the model
model = Sequential()
model.add(Embedding(max_words, 128, input_length=max_len))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(2, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Set up early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=10, batch_size=128, callbacks=[early_stop])

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)

print("Test accuracy:", accuracy)

Epoch 1/10
250/250 [==============================] - 26s 90ms/step - loss: 0.4157 - accuracy: 0.8042 - val_loss: 0.3115 - val_accuracy: 0.8659
Epoch 2/10
250/250 [==============================] - 22s 87ms/step - loss: 0.2456 - accuracy: 0.9005 - val_loss: 0.3435 - val_accuracy: 0.8661
Epoch 3/10
250/250 [==============================] - 22s 88ms/step - loss: 0.1701 - accuracy: 0.9348 - val_loss: 0.3604 - val_accuracy: 0.8584
Epoch 4/10
250/250 [==============================] - 22s 87ms/step - loss: 0.1024 - accuracy: 0.9642 - val_loss: 0.4518 - val_accuracy: 0.8526
Epoch 5/10
250/250 [==============================] - 22s 87ms/step - loss: 0.0590 - accuracy: 0.9793 - val_loss: 0.5730 - val_accuracy: 0.8471
Epoch 6/10
313/313 [==============================] - 5s 15ms/step - loss: 0.3132 - accuracy: 0.8674
Test accuracy: 0.8673999905586243


In [25]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping

# Split into train, test and validation sets
X_train, X_test, y_train, y_test = train_test_split(df['review'], df['sentiment'], test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Tokenize the data
max_words = 10000
max_len = 100
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
X_valid = tokenizer.texts_to_sequences(X_valid)

X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)
X_valid = pad_sequences(X_valid, maxlen=max_len)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_valid = to_categorical(y_valid)

# Build the model
model = Sequential()
model.add(Embedding(max_words, 128, input_length=max_len))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(2, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Set up early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=10, batch_size=16, callbacks=[early_stop])

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)

print("Test accuracy:", accuracy)

Epoch 1/10
2000/2000 [==============================] - 74s 36ms/step - loss: 0.3951 - accuracy: 0.8211 - val_loss: 0.3157 - val_accuracy: 0.8701
Epoch 2/10
2000/2000 [==============================] - 71s 35ms/step - loss: 0.2467 - accuracy: 0.9004 - val_loss: 0.3079 - val_accuracy: 0.8724
Epoch 3/10
2000/2000 [==============================] - 71s 36ms/step - loss: 0.1715 - accuracy: 0.9337 - val_loss: 0.3525 - val_accuracy: 0.8583
Epoch 4/10
2000/2000 [==============================] - 72s 36ms/step - loss: 0.1068 - accuracy: 0.9622 - val_loss: 0.3857 - val_accuracy: 0.8606
Epoch 5/10
2000/2000 [==============================] - 73s 36ms/step - loss: 0.0582 - accuracy: 0.9804 - val_loss: 0.5427 - val_accuracy: 0.8612
Epoch 6/10
2000/2000 [==============================] - 74s 37ms/step - loss: 0.0370 - accuracy: 0.9874 - val_loss: 0.6496 - val_accuracy: 0.8580
Epoch 7/10
313/313 [==============================] - 5s 14ms/step - loss: 0.3179 - accuracy: 0.8655
Test accuracy: 0.865499

## Transformer Based Models (DistilBERT and Roberta)

In [ ]:
import os
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
from sklearn.model_selection import train_test_split
import numpy as np

# Detect TPU and set up distribution strategy
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
tpu_strategy = tf.distribute.TPUStrategy(tpu)

model_name = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

max_len = 512
X_train = tokenizer(text=train['preprocessed_text'].tolist(), add_special_tokens=True, max_length=max_len, truncation=True, padding=True, return_tensors='tf')
y_train = train['label'].values

X_train_np = X_train['input_ids'].numpy()
X_train_np, X_val_np, y_train, y_val = train_test_split(X_train_np, y_train, test_size=0.2, random_state=42)

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_np, y_train)).batch(16)
val_dataset = tf.data.Dataset.from_tensor_slices((X_val_np, y_val)).batch(16)

with tpu_strategy.scope():
    model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

    optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

batch_size = 16
epochs = 10
history = model.fit(train_dataset, epochs=epochs, validation_data=val_dataset)

X_test = tokenizer(text=test['preprocessed_text'].tolist(), add_special_tokens=True, max_length=max_len, truncation=True, padding=True, return_tensors='tf')
y_test = test['label'].values

# Convert the test dataset into a TensorFlow dataset
test_dataset = tf.data.Dataset.from_tensor_slices((X_test['input_ids'], X_test['attention_mask'])).batch(16)

y_pred = model.predict(test_dataset)
y_pred_labels = np.argmax(y_pred.logits, axis=1)
accuracy = np.sum(y_pred_labels == y_test) / len(y_test)
print(f"Test accuracy: {accuracy:.2f}")

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_layer_norm', 'vocab_transform', 'vocab_projector', 'activation_13']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'pre_classifier', 'dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use i

Epoch 1/10


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


2000/2000 [==============================] - 181s 67ms/step - loss: 0.6414 - accuracy: 0.6095 - val_loss: 0.5745 - val_accuracy: 0.7329
Epoch 2/10
2000/2000 [==============================] - 127s 64ms/step - loss: 0.5298 - accuracy: 0.7581 - val_loss: 0.4805 - val_accuracy: 0.7713
Epoch 3/10
2000/2000 [==============================] - 127s 64ms/step - loss: 0.3901 - accuracy: 0.8346 - val_loss: 0.3480 - val_accuracy: 0.8493
Epoch 4/10
2000/2000 [==============================] - 128s 64ms/step - loss: 0.2702 - accuracy: 0.8927 - val_loss: 0.3187 - val_accuracy: 0.8779
Epoch 5/10
2000/2000 [==============================] - 128s 64ms/step - loss: 0.1982 - accuracy: 0.9278 - val_loss: 0.3694 - val_accuracy: 0.8443
Epoch 6/10
2000/2000 [==============================] - 127s 64ms/step - loss: 0.1436 - accuracy: 0.9513 - val_loss: 0.3727 - val_accuracy: 0.8627
Epoch 7/10
2000/2000 [==============================] - 127s 63ms/step - loss: 0.1353 - accuracy: 0.9566 - val_loss: 0.3637 - val

In [27]:
import os
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
from sklearn.model_selection import train_test_split
import numpy as np

# Detect TPU and set up distribution strategy
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
tpu_strategy = tf.distribute.TPUStrategy(tpu)

model_name = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

max_len = 512

# Split into train, test and validation sets
X_train, X_test, y_train, y_test = train_test_split(df['review'], df['sentiment'], test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

X_train = tokenizer(text=X_train.tolist(), add_special_tokens=True, max_length=max_len, truncation=True, padding=True, return_tensors='tf')
y_train = y_train.values

X_train_np = X_train['input_ids'].numpy()
X_train_np, X_val_np, y_train, y_val = train_test_split(X_train_np, y_train, test_size=0.2, random_state=42)

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_np, y_train)).batch(32)
val_dataset = tf.data.Dataset.from_tensor_slices((X_val_np, y_val)).batch(32)

with tpu_strategy.scope():
    model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

    optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

# Set up early stopping
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

batch_size = 32
epochs = 10
history = model.fit(train_dataset, epochs=epochs, validation_data=val_dataset, callbacks=[early_stop])

X_test = tokenizer(text=X_test.tolist(), add_special_tokens=True, max_length=max_len, truncation=True, padding=True, return_tensors='tf')
y_test = y_test.values

# Convert the test dataset into a TensorFlow dataset
test_dataset = tf.data.Dataset.from_tensor_slices((X_test['input_ids'], X_test['attention_mask'])).batch(32)

y_pred = model.predict(test_dataset)
y_pred_labels = np.argmax(y_pred.logits, axis=1)
accuracy = np.sum(y_pred_labels == y_test) / len(y_test)
print(f"Test accuracy: {accuracy:.2f}")

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'vocab_transform', 'activation_13', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'pre_classifier', 'dropout_27']
You should probably TRAIN this model on a down-stream task to be able to use i

Epoch 1/10


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


800/800 [==============================] - 123s 98ms/step - loss: 0.2685 - accuracy: 0.8887 - val_loss: 0.1984 - val_accuracy: 0.9181
Epoch 2/10
800/800 [==============================] - 72s 90ms/step - loss: 0.1389 - accuracy: 0.9500 - val_loss: 0.2203 - val_accuracy: 0.9162
Epoch 3/10
800/800 [==============================] - 71s 89ms/step - loss: 0.0960 - accuracy: 0.9668 - val_loss: 0.2421 - val_accuracy: 0.9191
Epoch 4/10
313/313 [==============================] - 16s 45ms/step
Test accuracy: 0.92


## Roberta Model

In [ ]:
import os
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
from sklearn.model_selection import train_test_split
import numpy as np

# Detect TPU and set up distribution strategy
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
tpu_strategy = tf.distribute.TPUStrategy(tpu)

model_name = 'roberta-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)

max_len = 512
X_train = tokenizer(text=train['text'].tolist(), add_special_tokens=True, max_length=max_len, truncation=True, padding=True, return_tensors='tf')
y_train = train['label'].values

X_train_np = X_train['input_ids'].numpy()
X_train_np, X_val_np, y_train, y_val = train_test_split(X_train_np, y_train, test_size=0.2, random_state=42)

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_np, y_train)).batch(16)
val_dataset = tf.data.Dataset.from_tensor_slices((X_val_np, y_val)).batch(16)

with tpu_strategy.scope():
    model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

    optimizer = tf.keras.optimizers.Adam(learning_rate=2e-6)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

batch_size = 16
epochs = 10

# Add early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(train_dataset, epochs=epochs, validation_data=val_dataset, callbacks=[early_stopping])

X_test = tokenizer(text=test['text'].tolist(), add_special_tokens=True, max_length=max_len, truncation=True, padding=True, return_tensors='tf')
y_test = test['label'].values

# Convert the test dataset into a TensorFlow dataset
test_dataset = tf.data.Dataset.from_tensor_slices((X_test['input_ids'], X_test['attention_mask'])).batch(16)

y_pred = model.predict(test_dataset)
y_pred_labels = np.argmax(y_pred.logits, axis=1)
accuracy = np.sum(y_pred_labels == y_test) / len(y_test)
print(f"Test accuracy: {accuracy:.2f}")

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
2000/2000 [==============================] - 324s 124ms/step - loss: 0.3419 - accuracy: 0.8178 - val_loss: 0.1628 - val_accuracy: 0.9371
Epoch 2/10
2000/2000 [==============================] - 239s 119ms/step - loss: 0.1485 - accuracy: 0.9466 - val_loss: 0.1449 - val_accuracy: 0.9467
Epoch 3/10
2000/2000 [==============================] - 238s 119ms/step - loss: 0.1236 - accuracy: 0.9557 - val_loss: 0.1463 - val_accuracy: 0.9479
Epoch 4/10
2000/2000 [==============================] - 238s 119ms/step - loss: 0.1065 - accuracy: 0.9628 - val_loss: 0.1471 - val_accuracy: 0.9485
Epoch 5/10
313/313 [==============================] - 19s 48ms/step
Test accuracy: 0.95
